En faisant le produit avec la matrice adaptative si jamais un pays se retrouve seul dans une classe, toute la colonne de cette classe se retrouve à zéro.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from adjency_matrix import AdaptativeMatrix, DTW,AgeReduction
import numpy as np
from sklearn.cluster import KMeans
import pandas as pd
from sklearn.decomposition import PCA

In [2]:
AM = AdaptativeMatrix()
dtw = DTW()
AR = AgeReduction()

In [24]:
def adjency_matrix(X: torch.Tensor, coord_countries: torch.Tensor):
    """
    X is 3D tensor
    coord_countries is 2D matrix contains longitude and latitude of each country. 1st column represents long and 2nd represents lat
    """
    n_countries  = X.shape[0]
    clusters = AM.adaptative_matrix(X)
    dtw_matrix = dtw.DTW_Matrix(X)
    long_lat_matrix = np.empty((n_countries, n_countries))
    for i in range(n_countries):
        for j in range(n_countries):
            long_lat_matrix[i,j] = torch.sqrt(torch.mean((coord_countries[i]-coord_countries[j])**2))
    
    # 
    adjency_mat = dtw_matrix * long_lat_matrix
    #print("adj_brut_matrix \n",adjency_mat)
    for i in range(n_countries):
        for j in range(n_countries):
            if clusters[i] != clusters[j]:
                adjency_mat[i,j] = 0.005
    
    #print("long_lat \n",long_lat_matrix)
    print("clusters \n",clusters)
    return adjency_mat


In [25]:
coord_countries = torch.rand(8,2)
X = torch.rand((8,5,3))

In [32]:
adjency_matrix(X,coord_countries)

The variance explained by principal components:  [0.75297127]
The variance explained by principal components:  [0.52076656]
The variance explained by principal components:  [0.76120859]
The variance explained by principal components:  [0.82852733]
The variance explained by principal components:  [0.76143834]
The variance explained by principal components:  [0.73838258]
The variance explained by principal components:  [0.40042795]
The variance explained by principal components:  [0.62651635]
The variance explained by principal components:  [0.75297127]
The variance explained by principal components:  [0.52076656]
The variance explained by principal components:  [0.76120859]
The variance explained by principal components:  [0.82852733]
The variance explained by principal components:  [0.76143834]
The variance explained by principal components:  [0.73838258]
The variance explained by principal components:  [0.40042795]
The variance explained by principal components:  [0.62651635]
clusters

tensor([[0.0000, 0.0050, 0.0050, 0.3926, 0.2782, 0.0050, 0.4142, 0.0050],
        [0.0050, 0.0000, 0.0050, 0.0050, 0.0050, 0.0050, 0.0050, 0.1196],
        [0.0050, 0.0050, 0.0000, 0.0050, 0.0050, 0.1616, 0.0050, 0.0050],
        [0.3926, 0.0050, 0.0050, 0.0000, 0.1840, 0.0050, 0.1971, 0.0050],
        [0.2782, 0.0050, 0.0050, 0.1840, 0.0000, 0.0050, 0.1523, 0.0050],
        [0.0050, 0.0050, 0.1616, 0.0050, 0.0050, 0.0000, 0.0050, 0.0050],
        [0.4142, 0.0050, 0.0050, 0.1971, 0.1523, 0.0050, 0.0000, 0.0050],
        [0.0050, 0.1196, 0.0050, 0.0050, 0.0050, 0.0050, 0.0050, 0.0000]],
       dtype=torch.float64)